## Loading Documents

there are some sub directories

each one has some .md documents

the first step is to load all documents in a list

each element of the list >> langchain document >> doc & meta data: document type

In [14]:
# import os
# import glob
# from langchain_community.document_loaders import DirectoryLoader, TextLoader


# path_ = "data\knowledge-base"

# folders = glob.glob(path_ + "\*")
# # print(folders)

# documents = []
# for folder  in folders:
#     sub_name = folder.split("\\")[-1]
    
#     # create a loader to load all documents inside each path
#     loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
#     folder_docs = loader.load()

#     for doc in folder_docs:
#         # add a new meta type with name "doc_type" which defines the sub directory (type of the document)
#         doc.metadata["doc_type"] = sub_name
#         documents.append(doc)

#         # break
        

# print(f"len of documents is {len(documents)}")
        
    

len of documents is 76


In [13]:
# documents

[Document(metadata={'source': 'data\\knowledge-base\\company\\about.md', 'doc_type': 'company'}, page_content="# About Insurellm\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.\n\nThe company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 200 employees with 12 offices across the US.\n\nHowever, the company underwent a strategic restructuring in 2022-2023 to focus on profitability and sustainable growth. This included consolidating office locations, implementing a remote-first strategy, and streamlining operations. As of 2025, Insurellm operates with a lean, highly efficient team of 32 employees who have 

In [17]:
# only documents of one subdirectory
import os
import glob
from langchain_community.document_loaders import DirectoryLoader, TextLoader

sub_path = "data\knowledge-base\company"

loader = DirectoryLoader(sub_path, glob = "*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
docs   = loader.load()
docs

[Document(metadata={'source': 'data\\knowledge-base\\company\\about.md'}, page_content="# About Insurellm\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.\n\nThe company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 200 employees with 12 offices across the US.\n\nHowever, the company underwent a strategic restructuring in 2022-2023 to focus on profitability and sustainable growth. This included consolidating office locations, implementing a remote-first strategy, and streamlining operations. As of 2025, Insurellm operates with a lean, highly efficient team of 32 employees who have built a portfolio of 32

In [18]:
len(docs)

4

In [2]:
# documents of all subdirectory
import os
import glob
from langchain_community.document_loaders import DirectoryLoader, TextLoader

root = "data\knowledge-base"
sub_roots = glob.glob(root + "\*")

documents = []
for sub_root in sub_roots:
    sub_name = sub_root.split("\\")[-1]
    loader = DirectoryLoader(sub_root, glob = "*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
    docs   = loader.load()
    # print(len(docs))

    for doc in docs:
        doc.metadata["doc_type"] = sub_name
        documents.append(doc)

    print(f"{sub_name} has {len(docs)} documents")


print(f"\ntotal number of documents {len(documents)}")


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


company has 4 documents
contracts has 32 documents
employees has 32 documents
products has 8 documents

total number of documents 76


In [3]:
doc

Document(metadata={'source': 'data\\knowledge-base\\products\\Rellm.md', 'doc_type': 'products'}, page_content="# Product Summary\n\n# Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Summary\n\nRellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility.\n\n## Features\n\n### AI-Driven Analytics\nRellm utilizes cutting-edge AI algorithms to provide predictive insights into risk exposures, enabling users to forecast trends and make informed decisions. Its real-time data analysis empowers reinsurance professionals with actionable i

### chunk 

split documents into chunks

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [5]:
len(chunks)

970

In [6]:
chunks[0]

Document(metadata={'source': 'data\\knowledge-base\\company\\about.md', 'doc_type': 'company'}, page_content='# About Insurellm\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.')

### Embedding and VectorDB


use ollama opensource model

"ollama pull nomic-embed-text"

In [7]:
# an embedding model and a vectorDB

## model
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

## VectorDB
from langchain_chroma import Chroma

DB_NAME = "./vector_db"

if os.path.exists(DB_NAME):
        Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents( documents=chunks, embedding=embeddings, persist_directory=DB_NAME )

# _collection is the internal Chroma object
collection = vectorstore._collection

# Count how many vectors exist  (should be as same as the num.chunks)
num_embedding = collection.count()
print(f"number of embedding vectors {num_embedding}")

number of embedding vectors 970


In [11]:
# get one sample from DB
sample = collection.get(limit = 1, include=["embeddings"])["embeddings"][0]

sample

array([-8.16939119e-03,  4.61893529e-02, -2.04238489e-01, -1.07331062e-02,
        6.74127340e-02,  2.91994028e-02,  1.12590455e-02, -5.15811257e-02,
        3.26927006e-02, -5.27352169e-02, -2.77016144e-02,  8.77187308e-03,
        9.69891176e-02, -2.04376597e-02, -2.89933104e-03,  5.64304355e-04,
       -2.42612883e-02,  8.42215202e-04, -6.46130517e-02,  4.82707284e-02,
       -1.99440289e-02, -6.87855408e-02, -6.31344318e-02, -4.36855704e-02,
        7.59502053e-02, -1.90560613e-02,  1.41621185e-02,  2.25346331e-02,
        4.33030650e-02,  1.72885247e-02,  6.73078522e-02, -1.55947926e-02,
       -2.61438638e-02,  3.65889934e-03, -3.19507532e-02,  1.80619806e-02,
       -2.61868094e-03,  3.89372930e-02,  3.24855279e-03,  4.57555242e-03,
        8.84064287e-03,  3.55053172e-02,  2.68706344e-02, -8.68975744e-02,
        4.16933186e-02,  3.28561924e-02,  1.78980604e-02,  4.35093082e-02,
        6.26166537e-02, -5.74720427e-02, -2.43191235e-03,  3.04489378e-02,
       -1.15935616e-02, -

In [ ]:
dimensions = len(sample)
print(dimensions)

768


## Ingestion

from loading documents into storing them into a vectorDB

In [16]:
# ===============================================================================================================
#                                               Loading Documents
# ===============================================================================================================

# documents of all subdirectory
import os
import glob
from langchain_community.document_loaders import DirectoryLoader, TextLoader

root = "data\knowledge-base"
sub_roots = glob.glob(root + "\*")

documents = []
for sub_root in sub_roots:
    sub_name = sub_root.split("\\")[-1]
    loader = DirectoryLoader(sub_root, glob = "*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
    docs   = loader.load()
    # print(len(docs))

    for doc in docs:
        doc.metadata["doc_type"] = sub_name
        documents.append(doc)

    print(f"{sub_name} has {len(docs)} documents")


print(f"\ntotal number of documents {len(documents)}")


# ===============================================================================================================
#                                               Documents into Chunk
# ===============================================================================================================

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"\nnum. chunks {len(chunks)}")

# ===============================================================================================================
#                                               Vectorize and Store
# ===============================================================================================================

# an embedding model and a vectorDB

## model
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

## VectorDB
from langchain_chroma import Chroma

DB_NAME = "./vector_db"

if os.path.exists(DB_NAME):
        Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents( documents=chunks, embedding=embeddings, persist_directory=DB_NAME )

# _collection is the internal Chroma object
collection = vectorstore._collection

# Count how many vectors exist  (should be as same as the num.chunks)
num_embedding = collection.count()
print(f"\nnumber of embedding vectors {num_embedding}")

assert len(chunks) == num_embedding

# get one sample from DB
sample = collection.get(limit = 1, include=["embeddings"])["embeddings"][0]

dimensions = len(sample)
print(f"dimensions : {dimensions}")

company has 4 documents
contracts has 32 documents
employees has 32 documents
products has 8 documents

total number of documents 76

num. chunks 970

number of embedding vectors 970
dimensions : 768
